# Before you start
Make sure we have access to GPU. We can use `nvidia-smi` command to do that. 

In [ ]:
!nvidia-smi

In [ ]:
import os
ROOT_DIR = os.path.abspath('./')
DATASET_DIR = os.path.join(ROOT_DIR, 'dataset')
RUN_DIR = os.path.join(ROOT_DIR, 'transfer', '3-pills-noaug')

# Check YOLOv8 installation

In [ ]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO
from IPython.display import display, Image

# Initialise Comet

In [ ]:
COMET_API_KEY = 'comet_api'
PROJECT = '3-pills'
WORKSPACE = 'pills-recognition'

In [ ]:
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model

## Create an experiment with your api key
experiment = Experiment(
    api_key=COMET_API_KEY,
    project_name=PROJECT,
    workspace=WORKSPACE,
    log_code=True,
    log_graph=True
)

# Train on custom 3-pills dataset

In [ ]:
# download dataset from roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="roboflow_api")
project = rf.workspace("-").project("-")
version = project.version(1)
dataset = version.download("yolov8")


# Hyperparameter Tuning

In [ ]:
# Initialise the YOLO model
model = YOLO('yolov8n.pt')

# Tune hyperparameters for 30 epochs 300 iterations
model.tune(
    data=os.path.join(DATASET_DIR, 'data.yaml'),
    epochs=30,
    iterations=300,
    optimizer='AdamW',
    plots=False,
    save=False,
    val=False,
    lr0=0.00841,
    lrf=0.0133,
    momentum=0.94996,
    weight_decay=0.00053,
    warmup_epochs=2.4783,
    warmup_momentum=0.88646,
    box=7.85252,
    cls=0.70948,
    dfl=1.44185,
    hsv_h=0.01481,
    hsv_s=0.7905,
    hsv_v=0.41703,
    degrees=0.0,
    translate=0.08783,
    scale=0.44492,
    shear=0.0,
    perspective=0.0,
    flipud=0.0,
    fliplr=0.63125,
    mosaic=1.0,
    mixup=0.0,
    copy_paste=0.0
)

## CLI

In [ ]:
!yolo task=detect \
mode=train \
model=yolov8s.pt \
data=path_to_data.yaml \
epochs=100 \
imgsz=256 \
lr0=0.00994 \
lrf=0.00915 \
momentum=0.919 \
weight_decay=0.00051 \
warmup_epochs=3.0 \
warmup_momentum=0.8 \
box=6.58183 \
cls=0.5228 \
dfl=1.62759 \
hsv_h=0.01385 \
hsv_s=0.71988 \
hsv_v=0.41728 \
degrees=0.0 \
translate=0.09693 \
scale=0.47275 \
shear=0.0 \
perspective=0.0 \
flipud=0.0 \
fliplr=0.5 \
mosaic=1.0 \
mixup=0.0 \
copy_paste=0.0

## Python

In [ ]:
PROJECT = 'mediseg_32pills_aug'

In [ ]:
# load a pretrained model
model = YOLO('yolov9e.pt') # yolov8n.pt, yolov8s.pt, yolov8m.pt, yolov8l.pt, yolov8x.pt
# model = YOLO('yolov8n.yaml')  # build a new model from YAML

# Train the model
results = model.train(
    data = os.path.join(DATASET_DIR, 'data.yaml'), 
    project = PROJECT, 
    epochs = 20, 
    patience = 5,
    save_period=1,
    save_json=True,
    imgsz = 256,
    lr0 = 0.00717,
    lrf = 0.01509,
    momentum = 0.90096,
    weight_decay = 0.00059,
    warmup_epochs = 2.51764,
    warmup_momentum = 0.83182,
    box = 6.61607,
    cls = 0.95173,
    dfl = 1.13263,
    hsv_h = 0.0155,
    hsv_s = 0.89727,
    hsv_v = 0.42929,
    degrees = 0.0,
    translate = 0.10899,
    scale = 0.28515,
    shear = 0.0,
    perspective = 0.0,
    flipud = 0.0,
    fliplr = 0.62413,
    mosaic = 0.89009,
    mixup = 0.0,
    copy_paste = 0.0
)

# # Train the model
# results = model.train(
#     data = os.path.join(DATASET_DIR, 'data.yaml'), 
#     project = PROJECT, 
#     epochs = 20, 
#     patience = 5,
#     save_period=1,
#     save_json=True,
#     imgsz = 256,
#     lr0 = 0.00717,
#     lrf = 0.01509,
#     momentum = 0.90096,
#     weight_decay = 0.00059,
#     warmup_epochs = 2.51764,
#     warmup_momentum = 0.83182,
#     box = 6.61607,
#     cls = 0.95173,
#     dfl = 1.13263,
#     hsv_h = 0.0,
#     hsv_s = 0.0,
#     hsv_v = 0.0,
#     degrees = 0.0,
#     translate = 0.0,
#     scale = 0.0,
#     shear = 0.0,
#     perspective = 0.0,
#     flipud = 0.0,
#     fliplr = 0.0,
#     mosaic = 0.0,
#     mixup = 0.0,
#     copy_paste = 0.0
# )


# Prediction

In [ ]:
# Load the best model
best_model = YOLO(os.path.join(RUN_DIR, 'train', 'weights', 'best.pt'))

test_path = os.path.join(DATASET_DIR, 'test', 'images')

# Make predictions on the test set
test_results = best_model.predict(
    source=test_path,  # Directory containing test images
    imgsz=256,  # Image size
    save_txt=True,  # Save results to text file
    save_conf=True,  # Save confidences in --save-txt labels
    save_dir=os.path.join(RUN_DIR, 'train8', 'predict')  # Directory to save results
)

import shutil

# Define the source and destination directories
source_dir = 'runs/detect/predict'
destination_dir = os.path.join('3-pills', 'train', 'predict')

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Move files from the source to the destination
for filename in os.listdir(source_dir):
    src_file = os.path.join(source_dir, filename)
    dst_file = os.path.join(destination_dir, filename)
    shutil.move(src_file, dst_file)

# Delete the source directory
shutil.rmtree(source_dir)

# End Comet

In [ ]:
experiment.end()